In [1]:
# Classification template

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder() # Encode countries
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) # Will encode the 3 country strings to 0,1,2

labelencoder_X_2 = LabelEncoder() # Encode gender
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) # Will encode country gender to 0,1


In [3]:
# Don't need to onehotencode gender because its binary

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

/Users/natepill/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/natepill/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [4]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [6]:
# Feature Scaling
# Very important for NN calculations
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.
/Users/natepill/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [8]:
# Initialize the Classifier ANN
classifier = Sequential()

# Adding input layer AND first hidden layer
# output_dim = (11 inputs nodes + 1 output node )/2 = 6
# rectifier activation

classifier.add(Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform"))
# Adding Dropout 
classifier.add(Dropout(p = 0.1))


'''
Dense parameters:
output_dim: number of nodes you want to add to the next HIDDEN layer
Determining the number of nodes in the hidden layer is an art, but one tip
could be to have the number of nodes in the hidden layer be equal to the average of the nodes in 
the input layer and output layer. You could also determine this number by doing some form of parameter tuning

init: Will randomly initizalize weights to 0, using the glorot_uniform function be default

activation: What activation function to use

input_dim: Number of input nodes in our INPUT layer, so we only need to specify this parameter ONCE
'''


'\nDense parameters:\noutput_dim: number of nodes you want to add to the next HIDDEN layer\nDetermining the number of nodes in the hidden layer is an art, but one tip\ncould be to have the number of nodes in the hidden layer be equal to the average of the nodes in \nthe input layer and output layer. You could also determine this number by doing some form of parameter tuning\n\ninit: Will randomly initizalize weights to 0, using the glorot_uniform function be default\n\nactivation: What activation function to use\n\ninput_dim: Number of input nodes in our INPUT layer, so we only need to specify this parameter ONCE\n'

In [9]:
# Adding second hidden layer

classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))
classifier.add(Dropout(p = 0.1))

# Adding output layer
# Sigmoid activation will output possibility of the classification
# We can use softmax activation function that is like sigmoid, but works for multiclass-classification
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

In [10]:
# Compiling the ANN
# Compiling is the "training" or "learning" process for our ANN
# Stochastic Gradient descent will utilze a logartimic loss function NOT MSE, this is based on the 
# sigmoid activation of our output layer
# loss function for binary logarithmic: binary_crossentropy, if not binary, then: categorical_crossentropy

classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
'''
Compile parameters:

optimizer: algorithm to find the optimial set of weights - How we tune our weights - ie: Adam Gradient Descent Algo.

loss: Loss function to determine the error between ypred and ytrue

metrics: list of metrics to be evaluated by the model and then improved upon, typically 'accuracy'

epochs: number of training iterations, one full cycle of foward and back propagations
'''


"\nCompile parameters:\n\noptimizer: algorithm to find the optimial set of weights - How we tune our weights - ie: Adam Gradient Descent Algo.\n\nloss: Loss function to determine the error between ypred and ytrue\n\nmetrics: list of metrics to be evaluated by the model and then improved upon, typically 'accuracy'\n\nepochs: number of training iterations, one full cycle of foward and back propagations\n"

In [11]:
# Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size = 10, epochs=100)

Epoch 1/100
7500/7500 [==============================] - 2s 243us/step - loss: 0.5130 - acc: 0.7949
Epoch 2/100
7500/7500 [==============================] - 1s 163us/step - loss: 0.4070 - acc: 0.8235
Epoch 3/100
7500/7500 [==============================] - 2s 210us/step - loss: 0.3725 - acc: 0.8492
Epoch 4/100
7500/7500 [==============================] - 1s 167us/step - loss: 0.3549 - acc: 0.8576 0s - loss: 0.3549 - acc: 0.
Epoch 5/100
7500/7500 [==============================] - 1s 167us/step - loss: 0.3488 - acc: 0.8581
Epoch 6/100
7500/7500 [==============================] - 1s 144us/step - loss: 0.3463 - acc: 0.8575
Epoch 7/100
7500/7500 [==============================] - 1s 164us/step - loss: 0.3450 - acc: 0.8601 0s - loss: 0.3432 - acc: 0.86
Epoch 8/100
7500/7500 [==============================] - 1s 168us/step - loss: 0.3437 - acc: 0.8601
Epoch 9/100
7500/7500 [==============================] - 1s 176us/step - loss: 0.3428 - acc: 0.8604 0s - loss: 
Epoch 10/100
7500/7500 [======

In [12]:
# Fitting classifier to the Training set

# Predicting the Test set results
y_pred = classifier.predict(X_test) # Returns ONLY probabilities of people leaving the bank

# Convert probabilities to True/False using a threshold value in order to evaluate the model. Threshold value will
# be dependent on the sensitivity of the results

y_pred = (y_pred > 0.5) # Compares each value to 0.5 threshold and replaces with bool


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [13]:
cm

array([[1887,  104],
       [ 248,  261]])

In [14]:
1923 + 183

2106

In [15]:
1923 + 183 + 68 + 326

2500

In [16]:
2106/2500 # Accuracy score

0.8424

## Evaluate the ANN

* We need a keras wrapper for scikit_learn's cross validation so that we can utilize cross valudation on our keras model

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [22]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform"))
    classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))
    classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
    classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size=10, epochs=100)

# Will contain all the accuracies generated by k-fold cross validation
# CV is the number of K-folds we want
# nj_jobs will allow us to do parallel computing so that we can train 10 k folds in a reasonable about of time,
# nj_jobs = -1 essentially tells the computer to use all available CPUs
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv=10, n_jobs= -1) 

KeyboardInterrupt: 

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()
mean

In [28]:
variance

0.019329194618068445

### Low bias!, low variance!

## Tuning ANN hyperparameters using GridSearchCV

* **Warning:** Grid Search w/ 10 k-folds will take hours to train

In [24]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

* If you want to tune hyperparameters that are in the architecture of your ANN, then you need to pass it along as function paramter

In [25]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform"))
    classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))
    classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
    classifier.compile(optimizer= optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

In [26]:
classifier = KerasClassifier(build_fn = build_classifier)

In [27]:
parameters = {'batch_size': [25, 32], 'epochs': [100, 250], 'optimizer': ['adam', 'rmsprop']}

In [28]:
# GridSearch hyperparameters w/ 10 folds
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)

### Running the cell below will take hours to train!

In [29]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100
6750/6750 [==============================] - 1s 110us/step - loss: 0.5768 - acc: 0.7945
Epoch 2/100
6750/6750 [==============================] - 0s 46us/step - loss: 0.4363 - acc: 0.7975
Epoch 3/100
6750/6750 [==============================] - 0s 49us/step - loss: 0.4294 - acc: 0.7975
Epoch 4/100
6750/6750 [==============================] - 0s 45us/step - loss: 0.4267 - acc: 0.7975
Epoch 5/100
6750/6750 [==============================] - 0s 41us/step - loss: 0.4242 - acc: 0.7975
Epoch 6/100
6750/6750 [==============================] - 0s 45us/step - loss: 0.4213 - acc: 0.7975
Epoch 7/100
6750/6750 [==============================] - 0s 45us/step - loss: 0.4191 - acc: 0.7975
Epoch 8/100
6750/6750 [==============================] - 0s 64us/step - loss: 0.4172 - acc: 0.8201
Epoch 9/100
6750/6750 [==============================] - 0s 48us/step - loss: 0.4160 - acc: 0.8222
Epoch 10/100
6750/6750 [==============================] - 0s 47us/step - loss: 0.4150 - acc: 0.8247
Epoch 11

6750/6750 [==============================] - 0s 47us/step - loss: 0.4001 - acc: 0.8357
Epoch 83/100
6750/6750 [==============================] - 0s 51us/step - loss: 0.4007 - acc: 0.8367
Epoch 84/100
6750/6750 [==============================] - 0s 48us/step - loss: 0.4001 - acc: 0.8344
Epoch 85/100
6750/6750 [==============================] - 0s 50us/step - loss: 0.4000 - acc: 0.8366
Epoch 86/100
6750/6750 [==============================] - 0s 47us/step - loss: 0.4002 - acc: 0.8369
Epoch 87/100
6750/6750 [==============================] - 0s 48us/step - loss: 0.4001 - acc: 0.8364
Epoch 88/100
6750/6750 [==============================] - 0s 49us/step - loss: 0.4002 - acc: 0.8364
Epoch 89/100
6750/6750 [==============================] - 0s 49us/step - loss: 0.4000 - acc: 0.8353
Epoch 90/100
6750/6750 [==============================] - 0s 48us/step - loss: 0.3998 - acc: 0.8379
Epoch 91/100
6750/6750 [==============================] - 0s 50us/step - loss: 0.3994 - acc: 0.8361
Epoch 92/100


6750/6750 [==============================] - 0s 53us/step - loss: 0.3389 - acc: 0.8622
Epoch 64/100
6750/6750 [==============================] - 0s 51us/step - loss: 0.3386 - acc: 0.8612
Epoch 65/100
6750/6750 [==============================] - 0s 63us/step - loss: 0.3387 - acc: 0.8628
Epoch 66/100
6750/6750 [==============================] - 1s 127us/step - loss: 0.3385 - acc: 0.8631
Epoch 67/100
6750/6750 [==============================] - 0s 64us/step - loss: 0.3381 - acc: 0.8601: 0s - loss: 0.3383 - acc: 0.8
Epoch 68/100
6750/6750 [==============================] - 0s 73us/step - loss: 0.3372 - acc: 0.8624
Epoch 69/100
6750/6750 [==============================] - 0s 68us/step - loss: 0.3382 - acc: 0.8622
Epoch 70/100
6750/6750 [==============================] - 0s 61us/step - loss: 0.3375 - acc: 0.8618
Epoch 71/100
6750/6750 [==============================] - 0s 61us/step - loss: 0.3378 - acc: 0.8621
Epoch 72/100
6750/6750 [==============================] - 0s 64us/step - loss: 0.3

6750/6750 [==============================] - 1s 78us/step - loss: 0.4017 - acc: 0.8354
Epoch 44/100
6750/6750 [==============================] - 0s 64us/step - loss: 0.4014 - acc: 0.8356: 0s - loss: 0.4041 - acc: 0.8
Epoch 45/100
6750/6750 [==============================] - 1s 95us/step - loss: 0.4013 - acc: 0.8341
Epoch 46/100
6750/6750 [==============================] - 0s 70us/step - loss: 0.4006 - acc: 0.8357: 0s - loss: 0.4025 - acc: 0.8
Epoch 47/100
6750/6750 [==============================] - 1s 93us/step - loss: 0.3999 - acc: 0.8353
Epoch 48/100
6750/6750 [==============================] - 0s 66us/step - loss: 0.4002 - acc: 0.8356
Epoch 49/100
6750/6750 [==============================] - 0s 70us/step - loss: 0.4001 - acc: 0.8330
Epoch 50/100
6750/6750 [==============================] - 1s 103us/step - loss: 0.3998 - acc: 0.8354
Epoch 51/100
6750/6750 [==============================] - 1s 76us/step - loss: 0.3992 - acc: 0.8350: 0s - loss: 0.4154 - acc: 0
Epoch 52/100
6750/6750 [

6750/6750 [==============================] - 0s 53us/step - loss: 0.4062 - acc: 0.8336
Epoch 24/100
6750/6750 [==============================] - 0s 50us/step - loss: 0.4056 - acc: 0.8345
Epoch 25/100
6750/6750 [==============================] - 0s 60us/step - loss: 0.4052 - acc: 0.8356
Epoch 26/100
6750/6750 [==============================] - 0s 52us/step - loss: 0.4054 - acc: 0.8348
Epoch 27/100
6750/6750 [==============================] - 0s 51us/step - loss: 0.4050 - acc: 0.8335
Epoch 28/100
6750/6750 [==============================] - 0s 50us/step - loss: 0.4047 - acc: 0.8353
Epoch 29/100
6750/6750 [==============================] - 0s 55us/step - loss: 0.4044 - acc: 0.8357
Epoch 30/100
6750/6750 [==============================] - 0s 52us/step - loss: 0.4044 - acc: 0.8324
Epoch 31/100
6750/6750 [==============================] - 0s 51us/step - loss: 0.4040 - acc: 0.8347
Epoch 32/100
6750/6750 [==============================] - 0s 53us/step - loss: 0.4044 - acc: 0.8332
Epoch 33/100


...

6750/6750 [==============================] - 0s 49us/step - loss: 0.3976 - acc: 0.8369
Epoch 226/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.3976 - acc: 0.8361
Epoch 227/250
6750/6750 [==============================] - 0s 50us/step - loss: 0.3976 - acc: 0.8357
Epoch 228/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.3977 - acc: 0.8361
Epoch 229/250
6750/6750 [==============================] - 0s 47us/step - loss: 0.3973 - acc: 0.8367
Epoch 230/250
6750/6750 [==============================] - 0s 47us/step - loss: 0.3975 - acc: 0.8367
Epoch 231/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.3971 - acc: 0.8345
Epoch 232/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.3976 - acc: 0.8366
Epoch 233/250
6750/6750 [==============================] - 0s 50us/step - loss: 0.3975 - acc: 0.8356
Epoch 234/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.3973 - acc: 0.8367
Epoc

6750/6750 [==============================] - 0s 48us/step - loss: 0.4040 - acc: 0.8345
Epoch 57/250
6750/6750 [==============================] - 0s 51us/step - loss: 0.4042 - acc: 0.8342
Epoch 58/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4040 - acc: 0.8345
Epoch 59/250
6750/6750 [==============================] - 0s 51us/step - loss: 0.4041 - acc: 0.8332
Epoch 60/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4040 - acc: 0.8327
Epoch 61/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4039 - acc: 0.8345
Epoch 62/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4039 - acc: 0.8330
Epoch 63/250
6750/6750 [==============================] - 0s 52us/step - loss: 0.4039 - acc: 0.8335
Epoch 64/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.4035 - acc: 0.8348
Epoch 65/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.4037 - acc: 0.8336
Epoch 66/250


6750/6750 [==============================] - 0s 48us/step - loss: 0.4017 - acc: 0.8354
Epoch 138/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4018 - acc: 0.8360
Epoch 139/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4014 - acc: 0.8359
Epoch 140/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.4019 - acc: 0.8359
Epoch 141/250
6750/6750 [==============================] - 0s 51us/step - loss: 0.4017 - acc: 0.8361
Epoch 142/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4017 - acc: 0.8344
Epoch 143/250
6750/6750 [==============================] - 0s 52us/step - loss: 0.4015 - acc: 0.8360
Epoch 144/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.4018 - acc: 0.8351
Epoch 145/250
6750/6750 [==============================] - 0s 49us/step - loss: 0.4018 - acc: 0.8361
Epoch 146/250
6750/6750 [==============================] - 0s 48us/step - loss: 0.4016 - acc: 0.8367
Epoc

6750/6750 [==============================] - 0s 68us/step - loss: 0.3955 - acc: 0.8353
Epoch 218/250
6750/6750 [==============================] - 0s 72us/step - loss: 0.3958 - acc: 0.8353
Epoch 219/250
6750/6750 [==============================] - 0s 69us/step - loss: 0.3956 - acc: 0.8347
Epoch 220/250
6750/6750 [==============================] - 0s 67us/step - loss: 0.3954 - acc: 0.8359
Epoch 221/250
6750/6750 [==============================] - 0s 71us/step - loss: 0.3957 - acc: 0.8357
Epoch 222/250
6750/6750 [==============================] - 0s 67us/step - loss: 0.3953 - acc: 0.8350
Epoch 223/250
6750/6750 [==============================] - 0s 72us/step - loss: 0.3954 - acc: 0.8350: 0s - loss: 0.3973 - acc: 0.
Epoch 224/250
6750/6750 [==============================] - 0s 70us/step - loss: 0.3953 - acc: 0.8363
Epoch 225/250
6750/6750 [==============================] - 0s 69us/step - loss: 0.3954 - acc: 0.8350: 0s - loss: 0.4029 - acc: 0.
Epoch 226/250
6750/6750 [======================

KeyboardInterrupt: 

In [ ]:
best_parameters = grid_search.best_parameters_
best_accuracy = grid_search.best_accuracy